In [3]:
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia/

^C


.zip: Skipping, found more recently modified local copy (use --force to force download)


### Aclaración


La organización de carpetas y la descompresión se hizo de manera manual, lo unico que se mantuvo en codigo fue la descarga de la primera celda. Se pretende poner los notebooks en una carpeta anterior a la carpeta que se geenera despues de descomprimir la descarga.

# Data Cleaning



In [1]:
def normalizar_programa(texto):
    if not isinstance(texto, str):
        return 'OTROS'

    texto = texto.lower()
    palabras_clave = {
        'admin': 'ADMINISTRACION',
        'contad': 'CONTADURIA',
        'ingenier': 'INGENIERIA',
        'medic': 'MEDICINA',
        'psicolog': 'PSICOLOGIA',
        'derech': 'DERECHO',
        'enfermer': 'ENFERMERIA',
        'econom': 'ECONOMIA',
        'arquitect': 'ARQUITECTURA'
    }

    for key, value in palabras_clave.items():
        if key in texto:
            return value

    return 'OTROS'

In [24]:
import pandas as pd
import numpy as np
def preprocess_data(df, is_test=False):

    # print(f"Registros originales en test: {len(df)}")

    # --- 1. Eliminación de columnas redundantes ---
    df = df.drop(['FAMI_TIENEINTERNET.1', 'ESTU_PRIVADO_LIBERTAD'], axis=1, errors='ignore')

    # --- 2. Conversión de variables booleanas con manejo de nulos mejorado ---
    bool_map = {'Si': 1, 'No': 0}
    bool_cols = ['FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
             'FAMI_TIENEAUTOMOVIL', 'ESTU_PAGOMATRICULAPROPIO',
             'FAMI_TIENECOMPUTADOR']

    # Convertir a numérico y mantener nulos
    for col in bool_cols:
        df[col] = df[col].map(bool_map)

    # --- 3. Procesamiento de estrato socioeconómico ---
    # Extraer números de estrato y convertir a float
    df['FAMI_ESTRATOVIVIENDA'] = (
        df['FAMI_ESTRATOVIVIENDA']
        .str.extract(r'(\d+)')
        .astype(float)
    )

    # Imputar con mediana (más robusto que moda)
    median_estrato = df['FAMI_ESTRATOVIVIENDA'].median()
    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].fillna(median_estrato)

    # Crear feature binaria para estratos bajos
    df['ESTRATO_BAJO'] = (df['FAMI_ESTRATOVIVIENDA'] <= 2).astype(int)

    # --- 4. Manejo de educación padres ---
    edu_mapping = {
        'Ninguno': 0,
        'No sabe': 0,
        'Primaria incompleta': 1,
        'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3,
        'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 4.5,
        'Técnica o tecnológica completa': 5,
        'Universitaria incompleta': 5.5,
        'Universitaria completa': 6,
        'Postgrado': 7
    }

    df['EDUCACION_PADRE'] = df['FAMI_EDUCACIONPADRE'].map(edu_mapping).fillna(0)
    df['EDUCACION_MADRE'] = df['FAMI_EDUCACIONMADRE'].map(edu_mapping).fillna(0)

    # Crear característica combinada
    df['EDUCACION_PADRES'] = (df['EDUCACION_PADRE'] + df['EDUCACION_MADRE']) / 2

    # --- 5. Manejo acceso a tecnología ---
    # Primero manejar nulos en las columnas booleanas
    tech_cols = ['FAMI_TIENEINTERNET', 'FAMI_TIENECOMPUTADOR']
    for col in tech_cols:
        # Convertir a flotante manteniendo nulos
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Imputar nulos con la moda
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val)

    # Ahora crear la característica combinada
    df['ACCESO_TECNOLOGIA'] = df['FAMI_TIENEINTERNET'] + df['FAMI_TIENECOMPUTADOR']

    # --- 6. Optimización de one-hot encoding para matrícula ---
    # Agrupar categorías
    matricula_groups = {
        'Menos de 500 mil': 'BAJA',
        'Entre 500 mil y menos de 1 millón': 'BAJA',
        'Entre 1 millón y menos de 2.5 millones': 'MEDIA',
        'Entre 2.5 millones y menos de 4 millones': 'MEDIA_ALTA',
        'Entre 4 millones y menos de 5.5 millones': 'ALTA',
        'Entre 5.5 millones y menos de 7 millones': 'ALTA',
        'Más de 7 millones': 'PREMIUM'
    }

    df['GRUPO_MATRICULA'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(matricula_groups).fillna('NO_PAGO')

    # Aplicar one-hot a grupos consolidados
    matricula_dummies = pd.get_dummies(df['GRUPO_MATRICULA'], prefix='MATRICULA')
    df = pd.concat([df, matricula_dummies], axis=1)

    # --- 7. Manejo de coeficientes ---
    # Eliminar coeficientes irrelevantes
    df = df.drop(['coef_3', 'coef_4'], axis=1, errors='ignore')

    # Crear interacción entre coeficientes relevantes
    df['COEF_INTERACCION'] = df['coef_1'] * df['coef_2']

    # --- 8. Limpieza final ---
    # Eliminar columnas redundantes
    drop_cols = [
        'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
        'GRUPO_MATRICULA', 'ESTU_PRGM_DEPARTAMENTO', 'PERIODO',  # Tambien se podria quitar tiene automovil y tiene lavadora, pero eso se hara más adelante.
        'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'
    ]
    df = df.drop(drop_cols, axis=1, errors='ignore')

    # Codificación de variable objetivo
    if not is_test and 'RENDIMIENTO_GLOBAL' in df.columns:
        rendimiento_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
        df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(rendimiento_map)

    df['PROGRAMA_AGRUPADO'] = df['ESTU_PRGM_ACADEMICO'].apply(normalizar_programa)
    df = df.drop('ESTU_PRGM_ACADEMICO', axis=1, errors='ignore')
    df = df.dropna(subset=['PROGRAMA_AGRUPADO']).copy()  # Elimina registros con None

    # Conservar solo programas con suficiente representación (top 15)
    top_programas = df['PROGRAMA_AGRUPADO'].value_counts().nlargest(15).index
    df['PROGRAMA_AGRUPADO'] = df['PROGRAMA_AGRUPADO'].apply(
        lambda x: x if x in top_programas else 'OTROS'
    )
    
    # One-Hot Encoding para mantener todos los registros
    df = pd.get_dummies(df, columns=['PROGRAMA_AGRUPADO'], prefix='PROGRAMA')
    # print(df.select_dtypes(include='object').columns)
    # print(f"Registros después de preprocesar: {len(df)}")
    
    if is_test:
        ids = df['ID'].copy()
        return df, ids
    else:
        return df


# Enfoque estructurado

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar datos de test
test_df = pd.read_csv('udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv')
df = pd.read_csv("udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv")

clean_df = preprocess_data(df,is_test=False)

# Separar características y objetivo
X = clean_df.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y = clean_df['RENDIMIENTO_GLOBAL']


# Entrenar scaler solo con columnas numéricas
numeric_cols = X.select_dtypes(include=['number']).columns
scaler = StandardScaler().fit(X[numeric_cols])

# Escalar datos de entrenamiento
X_scaled = X.copy()
X_scaled[numeric_cols] = scaler.transform(X[numeric_cols])


# Dividir en train y validation
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

Registros originales en test: 692500


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Modelo base
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=25,
    min_samples_split=10,
    class_weight='balanced_subsample',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# Evaluación
y_pred_rf = rf.predict(X_val)
print("Random Forest Accuracy:", accuracy_score(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))

# Optimización con RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [200, 300, 400],
    'max_depth': [20, 25, 30, None],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
rf_search.fit(X_train, y_train)

# Mejor modelo
best_rf = rf_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_val)
print("Optimized RF Accuracy:", accuracy_score(y_val, y_pred_best_rf))


Random Forest Accuracy: 0.3879494584837545
              precision    recall  f1-score   support

           0       0.40      0.50      0.44     34597
           1       0.30      0.24      0.27     34455
           2       0.30      0.22      0.26     34324
           3       0.49      0.58      0.53     35124

    accuracy                           0.39    138500
   macro avg       0.37      0.39      0.38    138500
weighted avg       0.37      0.39      0.38    138500



In [ ]:
# Cargar datos de test
test_df = pd.read_csv('udea-ai-4-eng-20251-pruebas-saber-pro-colombia/test.csv')

# Aplicar mismo preprocesamiento
X_test_processed, test_ids = preprocess_data(test_df, is_test=True)  # Usar misma función de preprocesamiento


# Escalar datos de test
X_test_scaled = X_test_processed.copy()
X_test_scaled[numeric_cols] = scaler.transform(X_test_processed[numeric_cols])

X_test_scaled_final = X_test_scaled.drop('ID', axis=1)

# Predecir
preds = best_rf.predict(X_test_scaled_final)
label_map = {0:'bajo', 1:'medio-bajo', 2:'medio-alto', 3:'alto'}
predicted_labels = [label_map[pred] for pred in preds]

# Crear submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predicted_labels
})
submission.to_csv('alternative_submission.csv', index=False)